In [2]:
from pathlib import Path
from typing import List, Dict
from collections import defaultdict
from subprocess import run


def find_subjects(base_path: Path, prefix: str = "OAS2_") -> List[Path]:
    return sorted([
        p for p in base_path.iterdir() if p.is_dir() and p.name.startswith(prefix)
    ])


def group_visits_by_subject(subject_dirs: List[Path]) -> Dict[str, List[Path]]:
    grouped = defaultdict(list)
    for path in subject_dirs:
        parts = path.name.split('_')
        if len(parts) >= 3:
            subject_id = parts[1]
            grouped[subject_id].append(path)
    return grouped


def get_qualified_subjects(subject_groups: Dict[str, List[Path]]) -> List[str]:
    qualified = []
    for subject_id, visits in subject_groups.items():
        scan_types = {v.name.split('_')[-1] for v in visits}
        if len(scan_types) >= 3 and {"MR1", "MR3"}.issubset(scan_types):
            qualified.append(subject_id)
    return sorted(qualified)


def run_fastsurfer_segmentation(setup_dir, image, subject_id, output_dir):
    setup_dir = Path(setup_dir)
    output_dir = Path(output_dir)

    command = f"""
{setup_dir}/run_fastsurfer.sh \
--t1 {image} \
--sd {output_dir} \
--sid {subject_id} \
--seg_only \
--py python3 \
--no_biasfield \
--no_cereb \
--no_hypothal \
--allow_root
"""
    print(f"Running FastSurfer for subject: {subject_id}")
    run(command, shell=True, check=True)


def process_subject_range(
    setup_dir: str,
    output_dir: str,
    data_dir: str,
    start: int,
    end: int
):
    data_dir = Path(data_dir)
    setup_dir = Path(setup_dir)
    output_dir = Path(output_dir)

    subject_visit_dirs = find_subjects(data_dir)
    subject_groups = group_visits_by_subject(subject_visit_dirs)
    qualified_subjects = get_qualified_subjects(subject_groups)

    # Print full list of qualified subjects with indices
    print("\n=== Qualified Subjects (with MR1 and MR3, at least 3 scans total) ===")
    for idx, sid in enumerate(qualified_subjects):
        print(f"[{idx}] OAS2_{sid}")
    print("=====================================================================")

    # Clamp range
    end = min(end, len(qualified_subjects))
    assigned_subjects = qualified_subjects[start:end]

    print(f"\n>>> Processing subjects {start} to {end - 1}:")
    for idx, sid in enumerate(assigned_subjects, start=start):
        print(f"  - [{idx}] OAS2_{sid}")

    for subject_id in assigned_subjects:
        visits = subject_groups[subject_id]
        for visit in visits:
            if visit.name.endswith("MR1") or visit.name.endswith("MR3"):
                t1_image_path = visit / "RAW/mpr-1.nifti.img"
                if not t1_image_path.exists():
                    print(f"Missing T1 image for {visit.name}")
                    continue
                run_fastsurfer_segmentation(setup_dir, t1_image_path, visit.name, output_dir)


In [3]:
# Person 1 does the first 5 subjects, with start=0, and end=5
# PLEASE update paths and start and end vals for your machine!
process_subject_range(
    setup_dir="/home/jovyan/FastSurfer",
    output_dir="/home/jovyan/fastsurfer_outputs",
    data_dir="/home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1",
    start=0,
    end=5
)



=== Qualified Subjects (with MR1 and MR3, at least 3 scans total) ===
[0] OAS2_0002
[1] OAS2_0005
[2] OAS2_0007
[3] OAS2_0012
[4] OAS2_0013
[5] OAS2_0017
[6] OAS2_0018
[7] OAS2_0020
[8] OAS2_0027
[9] OAS2_0031
[10] OAS2_0034
[11] OAS2_0036
[12] OAS2_0037
[13] OAS2_0040
[14] OAS2_0041
[15] OAS2_0044
[16] OAS2_0048
[17] OAS2_0049
[18] OAS2_0051
[19] OAS2_0057
[20] OAS2_0058
[21] OAS2_0061
[22] OAS2_0062
[23] OAS2_0064
[24] OAS2_0067
[25] OAS2_0070
[26] OAS2_0073
[27] OAS2_0076
[28] OAS2_0078
[29] OAS2_0079
[30] OAS2_0080
[31] OAS2_0090
[32] OAS2_0095

>>> Processing subjects 0 to 4:
  - [0] OAS2_0002
  - [1] OAS2_0005
  - [2] OAS2_0007
  - [3] OAS2_0012
  - [4] OAS2_0013
Running FastSurfer for subject: OAS2_0002_MR1
Setting ENV variable FASTSURFER_HOME to script directory /home/jovyan/FastSurfer. 
Change via environment to location of your choice if this is undesired (export FASTSURFER_HOME=/dir/to/FastSurfer)
time command failing, not using time...
Start of the log for a new run_fastsu

 79%|███████▉  | 203/256 [04:40<01:13,  1.38s/batch]
Traceback (most recent call last):
  File "/home/jovyan/FastSurfer/FastSurferCNN/run_prediction.py", line 801, in <module>
    sys.exit(main(**vars(_args)))
             ^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/FastSurfer/FastSurferCNN/run_prediction.py", line 704, in main
    pred_data = eval.get_prediction(
                ^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/FastSurfer/FastSurferCNN/run_prediction.py", line 403, in get_prediction
    pred_prob = model.run(pred_prob, image_name, orig_data, zoom, out=pred_prob)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/FastSurfer/FastSurferCNN/inference.py", line 467, in run
    out = self.eval(init_pred, test_data_loader, out=out, out_scale=out_res)
 

[ERROR: inference.py:  403]: Exception in batch 203 of coronal inference.
Traceback (most recent call last):
  File "/home/jovyan/FastSurfer/FastSurferCNN/inference.py", line 379, in eval
    pred = self.model(images, scale_factors, out_scale)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/FastSurfer/FastSurferCNN/models/networks.py", line 358, in forward
    decoder_output1 = super().forward(encoder_output0, scale_factor=scale_factor)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/FastSurfer/FastSu

KeyboardInterrupt: 

run_fastsurfer.sh terminated via signal at Tue, 05 Aug 2025 19:01:39 +0000!
ERROR: FastSurfer asegdkt segmentation failed.


In [ ]:
# Person 2 does the next 5 subjects, with start=5, and end=10
process_subject_range(
    setup_dir="/home/jovyan/my_proj/FastSurfer",
    output_dir="/home/jovyan/my_proj/fastsurfer_outputs",
    data_dir="/home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1",
    start=5,
    end=10
)
